In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import zipfile
import random
import pandas as dp
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip.extractall()
zip.close()

--2024-06-21 14:34:58--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.12.27, 142.250.65.123, 172.217.164.27, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.12.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.3’

10_food_classes_10_ 100%[===================>] 160.74M   119MB/s    in 1.4s    

2024-06-21 14:34:59 (119 MB/s) - ‘10_food_classes_10_percent.zip.3’ saved [168546183/168546183]



In [ ]:
for dirpath, dirnames, filenames in os.walk('10_food_classes_10_percent'):
  print(f'there are {len(dirnames)} directories, {len(filenames)} images in {dirpath}')

there are 2 directories, 0 images in 10_food_classes_10_percent
there are 10 directories, 0 images in 10_food_classes_10_percent/test
there are 0 directories, 250 images in 10_food_classes_10_percent/test/hamburger
there are 0 directories, 250 images in 10_food_classes_10_percent/test/chicken_wings
there are 0 directories, 250 images in 10_food_classes_10_percent/test/ramen
there are 0 directories, 250 images in 10_food_classes_10_percent/test/pizza
there are 0 directories, 250 images in 10_food_classes_10_percent/test/steak
there are 0 directories, 250 images in 10_food_classes_10_percent/test/grilled_salmon
there are 0 directories, 250 images in 10_food_classes_10_percent/test/ice_cream
there are 0 directories, 250 images in 10_food_classes_10_percent/test/fried_rice
there are 0 directories, 250 images in 10_food_classes_10_percent/test/chicken_curry
there are 0 directories, 250 images in 10_food_classes_10_percent/test/sushi
there are 10 directories, 0 images in 10_food_classes_10_p

In [ ]:
class_names = os.listdir('10_food_classes_10_percent/train')
len(class_names)

10

In [ ]:
train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')

test_data = test_datagen.flow_from_directory(test_dir,
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [ ]:
def tensorboard_callback(dir_name, exp_name):
  log_dir = dir_name +'/'+ exp_name + '/' + datetime.date.now().strftime(%Y%m%d)
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir)
  return tensorboard_callback


In [ ]:
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False)
base_model.trainable=False
input = tf.keras.layers.Input(shape=(224,224,3), name='input_layer')
X = base_model(input)
X = tf.keras.layers.GlobalAveragePooling2D(name='global_average_pooling')(X)
output = tf.keras.layers.Dense(10, activation='softmax', name='output_layer')(X)
model = tf.keras.Model(input, output)

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

history = model.fit(train_data,
          epochs=5,
          steps_per_epochs=len(train_data),
          validation_data=train_data,
          validation_steps=len(test_data),
          callbacks=[tensorboard_callback('Fine_tuning', 'Sequential_API')])

24274472/24274472 [==============================] - 0s 0us/step
